
Importing Libraries

In [ ]:
import re
import nltk
import string
import warnings
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt


Importing options.

In [ ]:
pd.set_option("display.max_colwidth", 200)
warnings.filterwarnings("ignore", category=DeprecationWarning)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
data = pd.read_csv('/content/drive/MyDrive/Explo/processed_mayank.csv', encoding = 'latin', header=0)

In [ ]:
data.head(10)

,Unnamed: 0,target,Message
0,0,0,upset update facebook texting might cry result school today also blah
1,1,0,dived many time ball managed save 50 rest go bound
2,2,0,whole body feel itchy like fire
3,3,0,behaving mad see
4,4,0,whole crew
5,5,0,need hug
6,6,0,hey long time see yes rain bit bit lol fine thanks hows
7,7,0,nope
8,8,0,que muera
9,9,0,spring break plain city snowing


In [ ]:
# data = data.rename(columns={0: 'target', 1: 'id', 2: 'date', 3: 'query', 4: 'username', 5: 'text', 6: 'tidy_tweet'})
data = data.rename(columns={0: 'id', 1: 'target', 'Message' : 'tidy_tweet' })

In [ ]:
# missing_data = data.isna().sum().sort_values(ascending=False)
# percentage_missing = round((data.isnull().sum()/data.isnull().count()).sort_values(ascending=False)*100,2)
# missing_info = pd.concat([missing_data,percentage_missing],keys=['Missing values','Percentage'],axis=1)
# missing_info.style.background_gradient()

In [ ]:
data.head(10)

,Unnamed: 0,target,tidy_tweet
0,0,0,upset update facebook texting might cry result school today also blah
1,1,0,dived many time ball managed save 50 rest go bound
2,2,0,whole body feel itchy like fire
3,3,0,behaving mad see
4,4,0,whole crew
5,5,0,need hug
6,6,0,hey long time see yes rain bit bit lol fine thanks hows
7,7,0,nope
8,8,0,que muera
9,9,0,spring break plain city snowing


In [ ]:
data['target'] = data['target'].replace(4,1)

In [ ]:
data["target"].value_counts()

1    800000
0    799999
Name: target, dtype: int64

In [ ]:
# Bag of words
from sklearn.feature_extraction.text import CountVectorizer
import gensim

In [ ]:
data['tidy_tweet']=data['tidy_tweet'].astype('str')

In [ ]:
# Building a model classification matrix
bow_vectorizer = CountVectorizer(max_df=0.90, min_df=2, max_features=1000, stop_words='english')
bow = bow_vectorizer.fit_transform(data['tidy_tweet'])
# x = bow.fit_transform(data['tidy_tweet'].apply(lambda x: np.str_(x)))
bow.shape

(1599999, 1000)

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score, accuracy_score

### Bag of words features

In [ ]:


# Extracting train and test BoW features

train_bow = bow[:1600000,:]
test_bow = bow[1600000:,:]
# print(train_bow.shape[0])
train_bow.shape[0]==data['target'].shape[0]
# # splitting data into training and validation set
xtrain_bow, xvalid_bow, ytrain, yvalid = train_test_split(train_bow, data['target'], random_state=42,test_size=0.3)

### XGB Classifier on training data(15m)

---



In [ ]:

from sklearn.metrics import f1_score, accuracy_score
from xgboost import XGBClassifier
xgb_model = XGBClassifier(max_depth=6, n_estimators=1000).fit(xtrain_bow, ytrain)
prediction = xgb_model.predict(xvalid_bow)
f1_score(yvalid, prediction)

0.7557303266518189

### **XBG Classifier on test data**

In [ ]:

# xgb_model = XGBClassifier(max_depth=6, n_estimators=1000).fit(xtrain_bow, ytrain)
prediction = xgb_model.predict(xtrain_bow)
f1_score(ytrain, prediction)

0.7598386994939006

CatBoost Model

In [ ]:
# !pip install catboost
# from catboost import CatBoostClassifier
# cb_model = CatBoostClassifier().fit(xtrain_bow, ytrain)
# prediction = cb_model.predict(xvalid_bow)
# accuracy_score(yvalid, prediction)